In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path
import os
from IPython.display import FileLink, display
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from scipy.stats import skew
from scipy.special import boxcox1p
import lightgbm as lgb

In [3]:
PATH = Path('./data')
PATH.mkdir(exist_ok=True)

# Home Credit Default Risk: feature engineering and RF benchark

This notebook walks through the Home Credit Default Risk dataset, attempting to find features that improve upon a RandomForest model.

## Download and extract dataset

In [2]:
!kaggle competitions download -c home-credit-default-risk --path={PATH}

sample_submission.csv.zip: Downloaded 117KB of 117KB to {PATH}
application_test.csv.zip: Downloaded 6MB of 6MB to {PATH}
application_train.csv.zip: Downloaded 34MB of 34MB to {PATH}
bureau.csv.zip: Downloaded 36MB of 36MB to {PATH}
bureau_balance.csv.zip: Downloaded 61MB of 61MB to {PATH}
previous_application.csv.zip: Downloaded 74MB of 74MB to {PATH}
credit_card_balance.csv.zip: Downloaded 94MB of 94MB to {PATH}
POS_CASH_balance.csv.zip: Downloaded 106MB of 106MB to {PATH}
installments_payments.csv.zip: Downloaded 267MB of 267MB to {PATH}
HomeCredit_columns_description.csv: Downloaded 37KB of 37KB to {PATH}


In [6]:
for file in os.listdir(PATH):
    if not file.endswith('zip'):
        continue
    
    file_path = PATH / file

    !unzip -f -q -d {PATH} {file_path}

## Application train

In [4]:
application_train = pd.read_csv(PATH / 'application_train.csv')

In [5]:
target = application_train.pop('TARGET')

In [6]:
application_test = pd.read_csv(PATH / 'application_test.csv')

In [7]:
all_data = pd.concat([application_train, application_test], ignore_index=True)

### Prepare columns

I'll start with the most basic processing, converting all categorical columns to Pandas categories. Any ordinal categorical columns will also be ordered with their categories.

In [8]:
columns = [
    dict(name='SK_ID_CURR', type='cont'),
    dict(name='NAME_CONTRACT_TYPE', type='cat'),
    dict(name='CODE_GENDER', type='cat'),
    dict(name='FLAG_OWN_CAR', type='cat'),
    dict(name='FLAG_OWN_REALTY', type='cat'),
    dict(name='CNT_CHILDREN', type='cat'),
    dict(name='AMT_INCOME_TOTAL', type='cont'),
    dict(name='AMT_CREDIT', type='cont'),
    dict(name='AMT_ANNUITY', type='cont'),
    dict(name='AMT_GOODS_PRICE', type='cont'),
    dict(name='NAME_TYPE_SUITE', type='cat'),
    dict(name='NAME_INCOME_TYPE', type='cat'),

    dict(
        name='NAME_EDUCATION_TYPE', type='ord', cats=[
            'Incomplete higher', 'Lower secondary',
            'Secondary / secondary special', 'Higher education',
            'Academic degree']),
    
    dict(name='NAME_FAMILY_STATUS', type='cat'),
    dict(name='NAME_HOUSING_TYPE', type='cat'),
    dict(name='REGION_POPULATION_RELATIVE', type='cont'),
    dict(name='DAYS_BIRTH', type='cont'),
    dict(name='DAYS_EMPLOYED', type='cont'),
    dict(name='DAYS_REGISTRATION', type='cont'),
    dict(name='DAYS_ID_PUBLISH', type='cont'),
    dict(name='OWN_CAR_AGE', type='cont'),
    dict(name='FLAG_MOBIL', type='ord', cats=[0, 1]),
    dict(name='FLAG_EMP_PHONE', type='ord', cats=[0, 1]),
    dict(name='FLAG_WORK_PHONE', type='ord', cats=[0, 1]),
    dict(name='FLAG_CONT_MOBILE', type='ord', cats=[0, 1]),
    dict(name='FLAG_PHONE', type='ord', cats=[0, 1]),
    dict(name='FLAG_EMAIL', type='ord', cats=[0, 1]),
    dict(name='OCCUPATION_TYPE', type='cat'),
    dict(name='CNT_FAM_MEMBERS', type='cont'),
    dict(name='REGION_RATING_CLIENT', type='ord', cats=[1, 2, 3]),
    dict(name='REGION_RATING_CLIENT_W_CITY', type='ord', cats=[1, 2, 3]),
    dict(name='WEEKDAY_APPR_PROCESS_START', type='cat'),
    dict(name='HOUR_APPR_PROCESS_START', type='cat'),
    dict(name='REG_REGION_NOT_LIVE_REGION', type='ord', cats=[0, 1]),
    dict(name='REG_REGION_NOT_WORK_REGION', type='ord', cats=[0, 1]),
    dict(name='LIVE_REGION_NOT_WORK_REGION', type='ord', cats=[0, 1]),
    dict(name='REG_CITY_NOT_LIVE_CITY', type='ord', cats=[0, 1]),
    dict(name='REG_CITY_NOT_WORK_CITY', type='ord', cats=[0, 1]),
    dict(name='LIVE_CITY_NOT_WORK_CITY', type='ord', cats=[0, 1]),
    dict(name='ORGANIZATION_TYPE', type='cat'),
    dict(name='EXT_SOURCE_1', type='cont'),
    dict(name='EXT_SOURCE_2', type='cont'),
    dict(name='EXT_SOURCE_3', type='cont'),
    dict(name='APARTMENTS_AVG', type='cont'),
    dict(name='BASEMENTAREA_AVG', type='cont'),
    dict(name='YEARS_BEGINEXPLUATATION_AVG', type='cont'),
    dict(name='YEARS_BUILD_AVG', type='cont'),
    dict(name='COMMONAREA_AVG', type='cont'),
    dict(name='ELEVATORS_AVG', type='cont'),
    dict(name='ENTRANCES_AVG', type='cont'),
    dict(name='FLOORSMAX_AVG', type='cont'),
    dict(name='FLOORSMIN_AVG', type='cont'),
    dict(name='LANDAREA_AVG', type='cont'),
    dict(name='LIVINGAPARTMENTS_AVG', type='cont'),
    dict(name='LIVINGAREA_AVG', type='cont'),
    dict(name='NONLIVINGAPARTMENTS_AVG', type='cont'),
    dict(name='NONLIVINGAREA_AVG', type='cont'),
    dict(name='APARTMENTS_MODE', type='cont'),
    dict(name='BASEMENTAREA_MODE', type='cont'),
    dict(name='YEARS_BEGINEXPLUATATION_MODE', type='cont'),
    dict(name='YEARS_BUILD_MODE', type='cont'),
    dict(name='COMMONAREA_MODE', type='cont'),
    dict(name='ELEVATORS_MODE', type='cat'),
    dict(name='ENTRANCES_MODE', type='cat'),
    dict(name='FLOORSMAX_MODE', type='cat'),
    dict(name='FLOORSMIN_MODE', type='cat'),
    dict(name='LANDAREA_MODE', type='cont'),
    dict(name='LIVINGAPARTMENTS_MODE', type='cont'),
    dict(name='LIVINGAREA_MODE', type='cont'),
    dict(name='NONLIVINGAPARTMENTS_MODE', type='cont'),
    dict(name='NONLIVINGAREA_MODE', type='cont'),
    dict(name='APARTMENTS_MEDI', type='cont'),
    dict(name='BASEMENTAREA_MEDI', type='cont'),
    dict(name='YEARS_BEGINEXPLUATATION_MEDI', type='cont'),
    dict(name='YEARS_BUILD_MEDI', type='cont'),
    dict(name='COMMONAREA_MEDI', type='cont'),
    dict(name='ELEVATORS_MEDI', type='cont'),
    dict(name='ENTRANCES_MEDI', type='cont'),
    dict(name='FLOORSMAX_MEDI', type='cont'),
    dict(name='FLOORSMIN_MEDI', type='cont'),
    dict(name='LANDAREA_MEDI', type='cont'),
    dict(name='LIVINGAPARTMENTS_MEDI', type='cont'),
    dict(name='LIVINGAREA_MEDI', type='cont'),
    dict(name='NONLIVINGAPARTMENTS_MEDI', type='cont'),
    dict(name='NONLIVINGAREA_MEDI', type='cont'),
    dict(name='FONDKAPREMONT_MODE', type='cat'),
    dict(name='HOUSETYPE_MODE', type='cat'),
    dict(name='TOTALAREA_MODE', type='cont'),
    dict(name='WALLSMATERIAL_MODE', type='cat'),
    dict(name='EMERGENCYSTATE_MODE', type='ord', cats=['No', 'Yes']),
    dict(name='OBS_30_CNT_SOCIAL_CIRCLE', type='cont'),
    dict(name='DEF_30_CNT_SOCIAL_CIRCLE', type='cont'),
    dict(name='OBS_60_CNT_SOCIAL_CIRCLE', type='cont'),
    dict(name='DEF_60_CNT_SOCIAL_CIRCLE', type='cont'),
    dict(name='DAYS_LAST_PHONE_CHANGE', type='cont'),
    dict(name='FLAG_DOCUMENT_2', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_3', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_4', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_5', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_6', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_7', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_8', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_9', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_10', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_11', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_12', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_13', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_14', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_15', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_16', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_17', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_18', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_19', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_20', type='ord', cats=[0, 1]),
    dict(name='FLAG_DOCUMENT_21', type='ord', cats=[0, 1]),
    dict(name='AMT_REQ_CREDIT_BUREAU_HOUR', type='cont'),
    dict(name='AMT_REQ_CREDIT_BUREAU_DAY', type='cont'),
    dict(name='AMT_REQ_CREDIT_BUREAU_WEEK', type='cont'),
    dict(name='AMT_REQ_CREDIT_BUREAU_MON', type='cont'),
    dict(name='AMT_REQ_CREDIT_BUREAU_QRT', type='cont'),
    dict(name='AMT_REQ_CREDIT_BUREAU_YEAR', type='cont')
]

In [9]:
all_data.DAYS_EMPLOYED = all_data.DAYS_EMPLOYED * -1
all_data.DAYS_REGISTRATION = all_data.DAYS_REGISTRATION * -1
all_data.DAYS_BIRTH = all_data.DAYS_BIRTH * -1

In [10]:
for col_desc in columns:
    col = col_desc['name']
    
    if col_desc['type'] == 'cat':
        all_data[col] = all_data[col].astype('category').cat.as_ordered()
    elif col_desc['type'] == 'ord':
        all_data[col] = (
            all_data[col].astype('category').cat.set_categories(
                col_desc['cats'], ordered=True))

In [11]:
print(all_data['ORGANIZATION_TYPE'].cat.categories)
print(all_data['FLAG_DOCUMENT_3'].cat.categories)

Index(['Advertising', 'Agriculture', 'Bank', 'Business Entity Type 1',
       'Business Entity Type 2', 'Business Entity Type 3', 'Cleaning',
       'Construction', 'Culture', 'Electricity', 'Emergency', 'Government',
       'Hotel', 'Housing', 'Industry: type 1', 'Industry: type 10',
       'Industry: type 11', 'Industry: type 12', 'Industry: type 13',
       'Industry: type 2', 'Industry: type 3', 'Industry: type 4',
       'Industry: type 5', 'Industry: type 6', 'Industry: type 7',
       'Industry: type 8', 'Industry: type 9', 'Insurance', 'Kindergarten',
       'Legal Services', 'Medicine', 'Military', 'Mobile', 'Other', 'Police',
       'Postal', 'Realtor', 'Religion', 'Restaurant', 'School', 'Security',
       'Security Ministries', 'Self-employed', 'Services', 'Telecom',
       'Trade: type 1', 'Trade: type 2', 'Trade: type 3', 'Trade: type 4',
       'Trade: type 5', 'Trade: type 6', 'Trade: type 7', 'Transport: type 1',
       'Transport: type 2', 'Transport: type 3', 'Transp

### Fill missing data

In [12]:
continuous_columns = [c['name'] for c in columns if c['type'] == 'cont']

In [13]:
with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000):
    display(all_data[continuous_columns].isnull().sum().sort_values(ascending=False))

COMMONAREA_MEDI                 248360
COMMONAREA_MODE                 248360
COMMONAREA_AVG                  248360
NONLIVINGAPARTMENTS_MEDI        246861
NONLIVINGAPARTMENTS_AVG         246861
NONLIVINGAPARTMENTS_MODE        246861
LIVINGAPARTMENTS_AVG            242979
LIVINGAPARTMENTS_MODE           242979
LIVINGAPARTMENTS_MEDI           242979
FLOORSMIN_MEDI                  241108
FLOORSMIN_AVG                   241108
YEARS_BUILD_MODE                236306
YEARS_BUILD_AVG                 236306
YEARS_BUILD_MEDI                236306
OWN_CAR_AGE                     235241
LANDAREA_AVG                    210844
LANDAREA_MEDI                   210844
LANDAREA_MODE                   210844
BASEMENTAREA_MEDI               207584
BASEMENTAREA_MODE               207584
BASEMENTAREA_AVG                207584
NONLIVINGAREA_AVG               195766
NONLIVINGAREA_MEDI              195766
NONLIVINGAREA_MODE              195766
EXT_SOURCE_1                    193910
ELEVATORS_MEDI           

For some columns it makes sense to deal with NAs by setting to 0.

In [14]:
all_data.loc[all_data['CNT_FAM_MEMBERS'].isnull(),'CNT_FAM_MEMBERS' ] = 0

For others, I'll impute the data by taking the median and add a `is_na` column.

In [15]:
for col_name, col in all_data[continuous_columns].items():
    if col.isnull().sum() > 0:
        all_data[f'{col_name}_is_na'] = col.isnull()
        all_data[col_name] = all_data[col_name].fillna(all_data[col_name].median())

### Numericalise

In [17]:
all_data_numeric = numericalise(all_data)

In [18]:
def rf_benchmark(inp, targs, num_splits=5, train_size=None):
    kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)
    model = RandomForestRegressor(n_jobs=-1, random_state=42)
    scores = cross_val_score(model, inp[:train_size], targs[:train_size], cv=kf, scoring='roc_auc')
    print(scores.mean())

In [20]:
rf_benchmark(all_data_numeric[:len(application_train)], target, train_size=10000)

0.6475092013010153


In [51]:
train_len = len(application_train)

## Previous application

* Count the previous applications.
* One-hot encode categories.
* Then average all the data.
* Deal with NaNs.

Thinking that feature combinations are going to be useful here. Might be nice to get a count of the number of loans rejected by type and so on. Might circle back to this depending on how important this feature is.

In [70]:
previous_application = pd.read_csv(PATH / 'previous_application.csv')

In [71]:
prev_app_counts = previous_application[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()

In [72]:
prev_app_cat_columns = [c for c in previous_application.columns if len(previous_application[c].unique()) < 50]

In [73]:
for col in prev_app_cat_columns:
    previous_application[col] = previous_application[col].astype('category').cat.as_ordered()

In [74]:
prev_app_dummies = pd.get_dummies(
    previous_application[prev_app_cat_columns], dummy_na=True)

In [75]:
previous_application.drop(prev_app_cat_columns, axis=1, inplace=True)
previous_application = pd.concat([previous_application, prev_app_dummies], axis=1)

In [76]:
previous_application = previous_application.drop(['SK_ID_PREV'], axis=1).groupby('SK_ID_CURR').mean()

In [77]:
previous_application = previous_application.join(prev_app_counts)

In [78]:
prev_app_cont_columns = [c for c in previous_application.columns if c not in prev_app_cat_columns]

In [79]:
# Replace NaNs with the median ()
for col_name, col in previous_application[prev_app_cont_columns].items():
    if col.isnull().sum() > 0:
        previous_application[f'{col_name}_is_na'] = col.isnull()
        previous_application[col_name] = previous_application[col_name].fillna(previous_application[col_name].median())

In [80]:
all_data_prev = all_data_numeric.join(previous_application, on='SK_ID_CURR', rsuffix='_PREV_APP')

In [81]:
for col_name, col in all_data_prev.items():
    if col.isnull().sum() > 0:
        all_data_prev[f'{col_name}_is_na'] = all_data_prev[col_name].isna()
        all_data_prev[col_name] = all_data_prev[col_name].fillna(all_data_prev[col_name].median())

In [82]:
rf_benchmark(all_data_prev[:train_len], target, train_size=10000)

0.6595987647394873


## Credit card balance

* One-hot encode categories.
* Take a mean of the various balances.

In [83]:
credit_card_balance = pd.read_csv(PATH / 'credit_card_balance.csv')

In [84]:
credit_card_balance = pd.concat([
    credit_card_balance.drop('NAME_CONTRACT_STATUS', axis=1),
    pd.get_dummies(credit_card_balance['NAME_CONTRACT_STATUS'], prefix='cc_bal_status_')
], axis=1)

In [85]:
cc_prevs = credit_card_balance[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()

In [86]:
credit_card_balance = credit_card_balance.drop('SK_ID_PREV', axis=1).join(cc_prevs, on='SK_ID_CURR')

In [87]:
credit_card_avgs = credit_card_balance.groupby('SK_ID_CURR').mean()

In [88]:
all_data_cc = all_data_prev.join(credit_card_avgs, on='SK_ID_CURR', rsuffix='_CC')

In [89]:
for col_name, col in all_data_cc.items():
    if col.isnull().sum() > 0:
        all_data_cc[f'{col_name}_is_na'] = all_data_cc[col_name].isna()
        all_data_cc[col_name] = all_data_cc[col_name].fillna(all_data_cc[col_name].median())

In [90]:
rf_benchmark(all_data_cc[:train_len], target, train_size=10000)

0.6529709555980182


## Credit bureau data

In [91]:
bureau = pd.read_csv(PATH / 'bureau.csv')

In [92]:
bureau_balance = pd.read_csv(PATH / 'bureau_balance.csv')

In [93]:
bureau_balance = pd.concat([
    bureau_balance,
    pd.get_dummies(bureau_balance.STATUS, prefix='bureau_bal')], axis=1).drop(
    'STATUS', axis=1)

In [94]:
month_balance_count = bureau_balance[['SK_ID_BUREAU', 'MONTHS_BALANCE']].groupby('SK_ID_BUREAU').count()

In [95]:
bureau_balance = bureau_balance.drop('MONTHS_BALANCE', axis=1).join(month_balance_count, on='SK_ID_BUREAU')

In [96]:
bureau_balance = bureau_balance.groupby('SK_ID_BUREAU').mean()

In [97]:
bureau_cat_columns = [b for b in bureau.columns if bureau[b].dtype == 'object']

In [98]:
bureau_dummies = pd.get_dummies(bureau[bureau_cat_columns], dummy_na=True)

In [99]:
bureau = pd.concat([bureau.drop(bureau_cat_columns, axis=1), bureau_dummies], axis=1)

In [100]:
bureau_joined = bureau.join(bureau_balance, on='SK_ID_BUREAU', rsuffix='bb')

In [101]:
num_bureau = bureau_joined[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby('SK_ID_CURR').count()

In [102]:
bureau_joined['SK_ID_BUREAU'] = bureau_joined['SK_ID_CURR'].map(num_bureau['SK_ID_BUREAU'])

In [103]:
avg_bureau = bureau_joined.groupby('SK_ID_CURR').mean()

In [104]:
all_data_bureau = all_data_cc.join(avg_bureau, on='SK_ID_CURR', rsuffix='_CC')

In [105]:
for col_name, col in all_data_bureau.items():
    if col.isnull().sum() > 0:
        all_data_bureau[f'{col_name}_is_na'] = all_data_bureau[col_name].isna()
        all_data_bureau[col_name] = all_data_bureau[col_name].fillna(
            all_data_bureau[col_name].median())

In [106]:
rf_benchmark(all_data_bureau[:train_len], target, train_size=10000)

0.6533579001522801


## Installment payments

In [107]:
install_payments = pd.read_csv('./data/installments_payments.csv')

In [108]:
install_prevs = install_payments[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()

In [109]:
install_payments = install_payments.drop('SK_ID_PREV', axis=1).join(install_prevs, on='SK_ID_CURR')

In [110]:
avg_install_payments = install_payments.groupby('SK_ID_CURR').mean()

In [111]:
all_data_ip = all_data_bureau.join(avg_install_payments, on='SK_ID_CURR', rsuffix='_IP')

In [112]:
for col_name, col in all_data_ip.items():
    if col.isnull().sum() > 0:
        all_data_ip[f'{col_name}_is_na'] = all_data_ip[col_name].isna()
        all_data_ip[col_name] = all_data_ip[col_name].fillna(all_data_ip[col_name].median())

In [113]:
rf_benchmark(all_data_ip[:train_len], target, train_size=10000)

0.6686260774977025


## Average POS

In [120]:
pos_cash_bal = pd.read_csv('./data/POS_CASH_balance.csv')

In [121]:
pos_cash_bal = pd.concat([pos_cash_bal, pd.get_dummies(pos_cash_bal.NAME_CONTRACT_STATUS)], axis=1)

In [122]:
num_prevs = pos_cash_bal[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()

In [123]:
pos_cash_bal['SK_ID_PREV'] = pos_cash_bal['SK_ID_CURR'].map(num_prevs['SK_ID_PREV'])
avg_pos_cash_bal = pos_cash_bal.groupby('SK_ID_CURR').mean()

In [124]:
all_data_pos = all_data_bureau.join(avg_pos_cash_bal, on='SK_ID_CURR', rsuffix='_POS')

In [125]:
for col_name, col in all_data_pos.items():
    if col.isnull().sum() > 0:
        all_data_pos[f'{col_name}_is_na'] = all_data_pos[col_name].isna()
        all_data_pos[col_name] = all_data_pos[col_name].fillna(
            all_data_pos[col_name].median())

In [126]:
rf_benchmark(all_data_pos[:train_len], target, train_size=10000)

0.6658425218879651


## Save DataFrame

In [129]:
all_data_pos.to_pickle(PATH / 'all_data_prepared.pkl')